In [2]:
import sys
sys.path.append('/content/drive/My Drive/Colab_Notebooks/Frame Prediction Pytorch')
from convlstm import ConvLSTM as ConvLstm           

In [3]:
import torch
import torch.nn as nn

x = torch.rand(8,5,3,256,256)

class convlstm_(nn.Module):  # can't make this a function in clas you don't have to give the input x
  def __init__(self,v,i):
    super().__init__()
    self.i = i 
    self.conv = ConvLstm(input_dim=v,hidden_dim=i,kernel_size=(1,1),num_layers=1,batch_first=True)
  def forward(self,x):          
    result = self.conv(x)[0][0]
    result = nn.BatchNorm3d(5)(result)
    result = nn.ReLU()(result) 
    result = self.conv(result)[0][0]
    result = nn.BatchNorm3d(5)(result)
    result = nn.ReLU()(result)
    result1 = nn.BatchNorm3d(5)(x)
    result = result+result1          
    return result               

In [4]:
import torch.nn as nn 
class TimeDistributed(nn.Module):
    def __init__(self,in_,out,kernel,stride):
        super().__init__()
        self.out = out
        module_list = [nn.Conv2d(in_channels=in_,out_channels=out,kernel_size=(kernel,kernel),stride = stride) for x in range(5)]
        self.layers = nn.ModuleList(module_list)
    
    def forward(self,x):
        output = torch.tensor([])
        batchsize,timestep,C,H,W = x.shape
        for c in range(timestep):
            output_ = self.layers[c](x[:,c,:,:,:])
            output_ = output_.unsqueeze(1)
            output = torch.cat((output,output_),1)
            output = nn.BatchNorm3d(output.shape[1])(output)
            output = nn.ReLU()(output)
            
        return output 

In [5]:
def ConvLstmBlock(in_,out):
  ui = [in_]*3
  seq = nn.Sequential(*[convlstm_(v,i) for v,i in zip(ui[:-1],ui[1:])])
  time_dis = TimeDistributed(in_,out,2,2)
  return nn.Sequential(*[seq,time_dis])

In [7]:
def ConvLstm64(x,outputs):
  input_list = [3,int(outputs/2),outputs]
  rr = nn.Sequential(*[ConvLstmBlock(inp,outp) for inp,outp in zip(input_list[:2],input_list[1:] )])
  return rr(x) 

In [8]:
x = torch.rand(8,5,3,256,256)

what = ConvLstm64(x,40)    

In [11]:
what.shape

torch.Size([8, 5, 40, 64, 64])